In [182]:
from PIL import Image

import imagehash

import requests

from bs4 import BeautifulSoup

from nltk.tokenize import word_tokenize

import string

from nltk.corpus import stopwords

def similarity(link):
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    text2=soup.find('div', { "class" : "field field--body" }).text 
    text=text2
    
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    #print(words)
    from nltk.stem.porter import PorterStemmer
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    #print(words)
    from sklearn.feature_extraction.text import CountVectorizer

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(words)
    Y=vectorizer.get_feature_names()
    from sklearn.decomposition import LatentDirichletAllocation

    number_of_topics = 1

    model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)
    model.fit(X)
    import pandas as pd
    def display_topics(model, feature_names, no_top_words):
        topic_dict = {}
        for topic_idx, topic in enumerate(model.components_):
            topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                            for i in topic.argsort()[:-no_top_words - 1:-1]]
            topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                            for i in topic.argsort()[:-no_top_words - 1:-1]]
        return pd.DataFrame(topic_dict)
    no_top_words = 20
    display_topics(model, Y, no_top_words)
    T1=display_topics(model, Y, no_top_words)
    return list(T1['Topic 0 words'])


In [192]:
first=similarity('https://wri-india.org/pt/blog/how-indian-companies-are-moving-quickly-carbon-pricing')


In [193]:
second=similarity('https://wri-india.org/pt/blog/going-local-climate-action')

In [195]:
res = len(set(first) & set(second)) / float(len(set(first) | set(second))) *100 
res


8.108108108108109

In [ ]:
print('the similarity between texts is',res,'%')

In [186]:
url = 'https://wri-india.org/sites/default/files/styles/large/public/Climate_Change.jpg?itok=MwJpVbYb/sites/all/themes/wri/images/logo.png'

page = requests.get(url)

f_ext = os.path.splitext(url)[-1]
f_name = 'img1{}'.format(f_ext)
with open(f_name, 'wb') as f:
    f.write(page.content)

In [187]:
url = 'https://wri-india.org/sites/default/files/styles/large/public/ashim-d-silva-unsplash.jpg'

page = requests.get(url)

f_ext = os.path.splitext(url)[-1]
f_namee = 'img2{}'.format(f_ext)
with open(f_namee, 'wb') as f:
    f.write(page.content)

In [188]:
import imagehash

hashh = imagehash.average_hash(Image.open('img1.png'))
otherhash = imagehash.average_hash(Image.open('img2.jpg'))
imgsim=100-(hashh - otherhash)
print(imgsim)

64


In [189]:
totalsim=abs((imgsim-res)*100/(imgsim+res))
totalsim

21.951219512195124

In [197]:
print('the blog is',res,'% similar in text and',imgsim,'% similar in imaging')
print('overall the blog is',totalsim,'% similar')

the blog is 8.108108108108109 % similar in text and 64 % similar in imaging
overall the blog is 21.951219512195124 % similar
